- 데이터를 MYSQL에서 불러온다 (E)
- 데이터 가공 (T)
- 데이터를 Postgres에 저장 (L)

이 작업을 우선 코드로 막 작성후 모듈화 해보자

In [ ]:
import dotenv
import os

# .env 파일 경로 찾기(자동으로 찾아줌)
env_path = dotenv.find_dotenv()

# .env 파일  (내용이 있으면 -> True, 없으면 -> False)
dotenv.load_dotenv(env_path)

from sqlalchemy import create_engine, text
import pymysql, psycopg2

class DBConnector:
    def __init__(self,engine_name,user,password,host,port,database):
            self.engine_name = engine_name
            self.user = user
            self.password = password
            self.host = host
            self.port = port
            self.database = database

    # PYMYSQL 연결 (방법 1: return으로 빼내는 방법)
    def pymysql_connection(self):
          pymysql_conn = pymysql.connect(
            user=self.user,
            password=self.password,
            host=self.host,
            port=int(self.port), # 몇년째 안고쳐주는 버그때문에 이렇게 해야함
            database=self.database,
            charset='utf8'
            )
          return pymysql_conn
    
    # PSYCOPG2 연결 (방법 2: self에 저장을 해서 뺴내기)
    def psycopg2_connection(self):
          self.psycopg2_conn = psycopg2.connect(
            user=self.user,
            password=self.password,
            host=self.host,
            port=self.port,
            dbname=self.database,
            )
          
    # SQLALCHEMY 연결
    def sqlalchemy_connection(self):
          sqlalchemy_conn = create_engine(f"{self.engine_name}://{self.user}:{self.password}@{self.host}:{self.port}/{self.database}")
          return sqlalchemy_conn
          

# 1) connection 생성
DB_SETTINGS = dict(
    mysql_params = dict(
        engine_name = os.getenv('MYSQL_ENGINE_NAME', ""),
        user = os.getenv('MYSQL_USER', ""),
        password = os.getenv('MYSQL_PASSWORD', ""),
        host = os.getenv('MYSQL_HOST', ""),
        port = os.getenv('MYSQL_PORT', ""),
        database = os.getenv('MYSQL_DATABASE', "")
    ),

    pg_params = dict(
        engine_name = os.getenv('PG_ENGINE_NAME', ""),
        user = os.getenv('PG_USER', ""),
        password = os.getenv('PG_PASSWORD', ""),
        host = os.getenv('PG_HOST', ""),
        port = os.getenv('PG_PORT', ""),
        database = os.getenv('PG_DATABASE', "")
    )
)

mysql_alchemy_conn = DBConnector(**DB_SETTINGS['mysql_params']).sqlalchemy_connection()
pg_alchemy_conn = DBConnector(**DB_SETTINGS['pg_params']).sqlalchemy_connection()
